# WIFI scanning with MacOS airport

LEGACY COMMANDS:  

Supported arguments:  
-c[[arg]] –channel=[[arg]] Set arbitrary channel on the card  
-z –disassociate Disassociate from any network  
-I –getinfo Print current wireless status, e.g. signal info, BSSID, port type etc.  
-s[[arg]] –scan=[[arg]] Perform a wireless broadcast scan.  

Will perform a directed scan if the optional [arg] is provided  
-x –xml Print info as XML  
-P –psk Create PSK from specified pass phrase and SSID.  

The following additional arguments must be specified with this command:  
–password=[arg] Specify a WPA password  
–ssid=[arg] Specify SSID when creating a PSK  
-h –help Show this help  

Configuring preferences (requires admin privileges)  
sudo airport en1 prefs JoinMode=Preferred RememberRecentNetworks=NO RequireAdmin=YES

https://developer.apple.com/library/archive/qa/qa1176/_index.html#//apple_ref/doc/uid/DTS10001707-CH1-SECNOTES

http://osxdaily.com/2007/01/18/airport-the-little-known-command-line-wireless-utility/

#### Show network

In [ ]:
!ifconfig

In [ ]:
!ifconfig en0

In [ ]:
!ifconfig | grep broadcast | arp -a

#### Scan local network for connected clients

arp: Address Resolution Protocol

In [ ]:
!ifconfig | grep broadcast | arp -a | grep '^?'

#### Scan local network for connected devices

In [ ]:
!arp -a | grep -v '^?'

In [ ]:
# !for ip in $(seq 1 254); do ping -c 1 192.168.0.$ip -W 1; done

### Scan wifi networks

In [ ]:
!/System/Library/PrivateFrameworks/Apple80211.framework/Versions/Current/Resources/airport en0 scan

#### Create symbolic link (in terminal)

Create a file link, like aliased shortcut, from `/usr/local/bin/airport` (in `$PATH`) to `/System/Library/PrivateFrameworks/Apple80211.framework/Versions/Current/Resources/airport`  
You can now just use `airport` to execute the file

```bash
sudo ln -s /System/Library/PrivateFrameworks/Apple80211.framework/Versions/Current/Resources/airport /usr/local/bin/airport
password🗝:
```

#### Scan wifi networks

In [ ]:
hotspots = !airport scan
hotspots

In [ ]:
!airport -s

In [ ]:
!airport -s | grep NONE

#### Show host router details

In [ ]:
!airport -I

In [ ]:
!sudo airport sniff 6 -AllUserland

#### Monitor Wi-Fi Signal Strength from Command Line on Mac OS X and View RSSI History

In [ ]:
!while x=1; do airport -I | grep CtlRSSI; sleep 0.5; done

In [ ]:
!while x=1; do airport -I | grep CtlRSSI | sed -e 's/^.*://g' | xargs -I SIGNAL printf "\rRSSI dBm: SIGNAL"; sleep 0.5; done

#### Scan WIFI networks every 5 secs

In [ ]:
!while x=1; do airport -s; sleep 0.5; done

#### Sniff packets on channel

Goto terminal and start sniffer.  
All packets will be saved in //tmp/airportSniffXXXXXX.cap file, which can then be read with scapy.  
Stop sniffer with control-c.  

```bash
sudo airport sniff 6
password🗝:
```

#### Read pcap sniffer file

In [ ]:
from scapy.all import *
sniff_file = '../../../../tmp/{}'.format('airportSniffG5T3A4.cap')
!head $sniff_file
pkts = rdpcap(sniff_file)
pkts

#### Read pcap file with tcpdump

http://osxdaily.com/2012/02/28/find-scan-wireless-networks-from-the-command-line-in-mac-os-x/

In [ ]:
!tcpdump -r $sniff_file | grep -i reassoc

In [ ]:
!tcpdump -r $sniff_file | less

In [ ]:
!tcpdump -r $sniff_file | grep oui

In [ ]:
!tcpdump -r $sniff_file

In [ ]:
def read_pcap(sniff_file):
    """Read .pcap file from airport sniff <channel>.
    https://www.programcreek.com/python/example/103591/scapy.all.rdpcap"""
    pkts = rdpcap(sniff_file)
    
    for pkt in pkts:
        if pkt.haslayer(Dot11):
#             print(dir(pkt))
            print('pkt.type: ', pkt.type, pkt.addr2)
#             print('pkt.getlayer: ', pkt.getlayer)
#             print(pkt[RadioTap], pkt[Dot11], pkt[Dot11QoS], pkt[Dot11WEP])

            if pkt.type == 0 and pkt.subtype == 8 :
                print('Available SSID: {} mac: {}'.format(pkt.info, pkt.addr2))
            
            # monitor ARP
            if ARP in pkt and pkt[ARP].op in (1,2): #who-has or is-at
                print(pkt.sprintf("%ARP.hwsrc% %ARP.psrc%"))

read_pcap(sniff_file)

In [ ]:
def print_highest_layer(sniff_file):
    """"""
    pkts = rdpcap(sniff_file)
    for pkt in pkts:
        try:
            print(pkt.lastlayer)
        except:
            continue
    
print_highest_layer(sniff_file)

In [ ]:
def get_key_in(pcap_file):
    """ Extract keyboard input reports from a pcap file. 
     The extraction depends only on the size of packets. """
    
    pcap = rdpcap(pcap_file)
    in_reports = []
    
    for p in pcap:
        if len(p) == 35:
            in_reports.append(p.load[-8:])
            
    return in_reports 

get_key_in(sniff_file)

In [ ]:
def read_pcap(filename):
    """
    @param filename: Filesystem path to the pcap.

    Returns:
      [{"client": "\x17\x52\x15"}, {"server": "\x17\x15\x13"}]
    """
    from scapy.all import IP, Raw, rdpcap

    packets = rdpcap(filename)

    checking_first_packet = True
    client_ip_addr = None
    server_ip_addr = None

    ssl_packets = []
    messages = []

    """
    pcap assumptions:

    pcap only contains packets exchanged between a Tor client and a Tor
    server.  (This assumption makes sure that there are only two IP addresses
    in the pcap file)

    The first packet of the pcap is sent from the client to the server. (This
    assumption is used to get the IP address of the client.)

    All captured packets are TLS packets: that is TCP session
    establishment/teardown packets should be filtered out (no SYN/SYN+ACK)
    """

    """
    Minimally validate the pcap and also find out what's the client
    and server IP addresses.
    """
    for packet in packets:
        if checking_first_packet:
            client_ip_addr = packet[IP].src
            checking_first_packet = False
        else:
            if packet[IP].src != client_ip_addr:
                server_ip_addr = packet[IP].src

        try:
            if (packet[Raw]):
                ssl_packets.append(packet)
        except IndexError:
            pass

    """Form our list."""
    for packet in ssl_packets:
        if packet[IP].src == client_ip_addr:
            messages.append({"client": str(packet[Raw])})
        elif packet[IP].src == server_ip_addr:
            messages.append({"server": str(packet[Raw])})
        else:
            raise("Detected third IP address! pcap is corrupted.")

    return messages 

read_pcap(sniff_file)

In [ ]:
def cap_session(pcap_path):
    capture = rdpcap(pcap_path)  # TODO when go live change to session capture
    first = True
    curr_session = None
    session_info = [0, ] * 3
    for pkt in capture:
        if not pkt.haslayer(TCP) and not pkt.haslayer(IP) and pkt.len <= 0:
            continue
        
        if first:
            first = False
            if is_client(pkt):
                session_info[0] = pkt[IP].src
                session_info[1] = pkt[IP].dst
                session_info[2] = "TCP"
                curr_session = Session(pkt, session_info, session_info[0])
            else:
                session_info[0] = pkt[IP].dst
                session_info[1] = pkt[IP].src
                session_info[2] = "TCP"
                curr_session = Session(pkt, session_info, session_info[0])
        else:
            curr_session.update_session(pkt)

    return curr_session 

cap_session(sniff_file)

Theif fm.haslayer(Dot11):statementislikeafilter,whichpassesonlythe Dot11traffic;Dot11indicates802.11traffic.Thenextif((fm.type == 0) & (fm. subtype==8)): statementisanotherfilter,whichpassestrafficwherethe

frametype is 0 and the frame subtype is 8; 
type 0 represents the management frame and subtype 8representsthebeaconframe.
Inthenextline,theif fm.addr2 not in ap_list: statement is used to remove the redundancy; if AP's MAC address is not in ap_list, then it appends the list and adds the address to the list as stated in the next line. The next line prints the output. 

You might want to obtain all the clients of a particular AP. In this situation, you have to capture the probe request frame. In scapy, this is called Dot11ProbeReq.

The probe request frame contains some interesting information such as the source address and SSID, as highlighted in the preceding screenshot.

“The Info field in the 802.11 Beacon Frame typically contains the name of the hidden network. In hidden network”


In [ ]:
import struct

def sniffDot11(p):
    if p.haslayer(Dot11Beacon):
        bcon = p.getlayer(Dot11Beacon).info
        print(struct.pack('B'*len(bcon), *bcon).decode('utf8'), p.getlayer(Dot11).addr2)
        if p.getlayer(Dot11Beacon).info == '':
            addr2 = p.getlayer(Dot11).addr2
            if addr2 not in hiddenNets:
                print('Detected Hidden SSID: {} with MAC: {}'.format(bcon_, addr2))

sniff(offline=sniff_file, prn=sniffDot11, store=0)

In [ ]:
import sys
import struct

hiddenNets = []
unhiddenNets = []

def sniffDot11(p):
    """Show hidden network SSID"""
    if p.haslayer(Dot11ProbeResp):
        addr2 = p.getlayer(Dot11).addr2
        netName = p.getlayer(Dot11ProbeResp).info
#         print(netName)
        print(struct.pack('B'*len(netName), *netName).decode('latin1', errors='strict'))
        if (addr2 in hiddenNets) & (addr2 not in unhiddenNets):
            netName = p.getlayer(Dot11ProbeResp).info
            print('[+] Decloaked Hidden SSID: {} for MAC: {} '.format(netName, addr2))
            unhiddenNets.append(addr2)
    if p.haslayer(Dot11Beacon):
        if p.getlayer(Dot11Beacon).info == '':
            addr2 = p.getlayer(Dot11).addr2
            if addr2 not in hiddenNets:
                print('[-] Detected Hidden SSID: with MAC: {}'.format(addr2))
                hiddenNets.append(addr2)


sniff(offline=sniff_file, prn=sniffDot11)

In [ ]:
from scapy.all import *
import struct

probe_req = []
ap_name = 'appelflap' #input("Please enter the AP name ")

def probesniff(fm):
    """"""
    if fm.haslayer(Dot11ProbeReq):
        netName = fm.getlayer(Dot11ProbeReq).info
        client_name = struct.pack('256s', netName).decode('latin1')
        print(client_name, fm.addr2)
        if client_name == ap_name:
            if fm.addr2 not in probe_req:
                print("New Probe Request: ", client_name)
                print("MAC ", fm.addr2)
                probe_req.append(fm.addr2)
    
sniff(offline=sniff_file, prn=probesniff, store=0)
probe_req

In [ ]:
i = 1
def info(fm):
    """Detect authentication attempts/attacks
    subtype 12: deauth frame
    """
    if fm.haslayer(Dot11):
        if ((fm.type==0) & (fm.subtype==12)):
            global i
            print(fm.subtype) #(dir(fm))
#             client_name = fm.info | 'none'
            print("Deauth detected #{}, client mac: {}, {}".format(i, fm.addr2, fm.getlayer(Dot11).info))
            i=i+1
            
sniff(offline=sniff_file, prn=info, store=0)

In [ ]:
import pyshark

In [ ]:
cap = pyshark.FileCapture(sniff_file, only_summaries=True)
dir(cap)
# cap[0]['WLAN']._field_names

In [ ]:
pkts = rdpcap(sniff_file)
sessions = pkts.sessions()
for session in sessions:
    print(sessions)
    for packet in sessions[session]:
#         print(packet)
        try:
            if packet[TCP].dport == 80 or packet[TCP].sport == 80:
                print(packet[TCP].payload)
        except:
            pass

#### Scapy - sniff offline

In [ ]:
def packetHandler(pkt):
    try:
        if pkt.haslayer(Dot11):
            if pkt.type == 0 and pkt.subtype == 8 :
                print("Available SSID: {} wlan: {}".format(pkt.info, pkt.addr2))
    except:
        pass

sniff(offline=sniff_file, prn=packetHandler, store=0)

In [ ]:
def pktIdentifier(pkt):
    if pkt.haslayer(Dot11Beacon):
        print("[+] Detected 802.11 Beacon Frame")
    elif pkt.haslayer(Dot11ProbeReq):
        print("[+] Detected 802.11 Probe Frame")

sniff(offline=sniff_file, prn=pktIdentifier, store=0)

In [ ]:
probeReqs = []
def sniffProbe(p):
    if p.haslayer(Dot11ProbeReq):
        netName = p.getlayer(Dot11ProbeReq).info
        if netName not in probeReqs:
            probeReqs.append(netName)
#             print(type(netName))
            try:
#                 print(struct.pack('p'*len(netName), netName).decode('utf8', errors='ignore'))
                print('struct', struct.pack('!s', netName).decode('utf8', errors='ignore'))
                print(len(netName))
                print('[+] Detected New Probe Request: ' + netName.decode(encoding="utf-8", errors="ignore"))
                print('[+] Detected New Probe Request: ' + netName.decode(encoding="latin1", errors="ignore"))#decode('utf-8'), .decode('latin1')
    #             print(binascii.hexlify(bytes(packet[TCP].payload)))
            except:
                pass
            
sniff(offline=sniff_file, prn=sniffProbe)